In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv(
    "../data/household_power_consumption.txt",
    sep=";",
    na_values="?",
    low_memory=False
)

print(df.shape)
df.head()


(2075259, 9)


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [3]:
df["datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    dayfirst=True
)

df.set_index("datetime", inplace=True)
df.drop(["Date", "Time"], axis=1, inplace=True)

df = df.ffill()
df.head()


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [4]:
hourly_df = df.resample("H").mean()

print(hourly_df.shape)
hourly_df.head()


(34589, 7)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:00:00,4.222889,0.229000,234.643889,18.100000,0.0,0.527778,16.861111
2006-12-16 18:00:00,3.632200,0.080033,234.580167,15.600000,0.0,6.716667,16.866667
2006-12-16 19:00:00,3.400233,0.085233,233.232500,14.503333,0.0,1.433333,16.683333
2006-12-16 20:00:00,3.268567,0.075100,234.071500,13.916667,0.0,0.000000,16.783333
2006-12-16 21:00:00,3.056467,0.076667,237.158667,13.046667,0.0,0.416667,17.216667


In [5]:
hourly_df["total_sub_metering"] = (
    hourly_df["Sub_metering_1"] +
    hourly_df["Sub_metering_2"] +
    hourly_df["Sub_metering_3"]
)

hourly_df[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3", "total_sub_metering"]].head()


,Sub_metering_1,Sub_metering_2,Sub_metering_3,total_sub_metering
datetime,,,,
2006-12-16 17:00:00,0.0,0.527778,16.861111,17.388889
2006-12-16 18:00:00,0.0,6.716667,16.866667,23.583333
2006-12-16 19:00:00,0.0,1.433333,16.683333,18.116667
2006-12-16 20:00:00,0.0,0.000000,16.783333,16.783333
2006-12-16 21:00:00,0.0,0.416667,17.216667,17.633333


In [6]:
hourly_df["hour"] = hourly_df.index.hour
hourly_df["day"] = hourly_df.index.day
hourly_df["month"] = hourly_df.index.month
hourly_df["weekday"] = hourly_df.index.weekday

hourly_df[["hour", "day", "month", "weekday"]].head()


,hour,day,month,weekday
datetime,,,,
2006-12-16 17:00:00,17,16,12,5
2006-12-16 18:00:00,18,16,12,5
2006-12-16 19:00:00,19,16,12,5
2006-12-16 20:00:00,20,16,12,5
2006-12-16 21:00:00,21,16,12,5


In [7]:
hourly_df["gap_lag_1"] = hourly_df["Global_active_power"].shift(1)
hourly_df["gap_lag_24"] = hourly_df["Global_active_power"].shift(24)

hourly_df[["Global_active_power", "gap_lag_1", "gap_lag_24"]].head(30)


,Global_active_power,gap_lag_1,gap_lag_24
datetime,,,
2006-12-16 17:00:00,4.222889,NaN,NaN
2006-12-16 18:00:00,3.632200,4.222889,NaN
2006-12-16 19:00:00,3.400233,3.632200,NaN
2006-12-16 20:00:00,3.268567,3.400233,NaN
2006-12-16 21:00:00,3.056467,3.268567,NaN
2006-12-16 22:00:00,2.200133,3.056467,NaN
2006-12-16 23:00:00,2.061600,2.200133,NaN
2006-12-17 00:00:00,1.882467,2.061600,NaN
2006-12-17 01:00:00,3.349400,1.882467,NaN


In [8]:
hourly_df["gap_roll_mean_24"] = (
    hourly_df["Global_active_power"].rolling(window=24).mean()
)

hourly_df["gap_roll_std_24"] = (
    hourly_df["Global_active_power"].rolling(window=24).std()
)

hourly_df[["gap_roll_mean_24", "gap_roll_std_24"]].head(30)


,gap_roll_mean_24,gap_roll_std_24
datetime,,
2006-12-16 17:00:00,NaN,NaN
2006-12-16 18:00:00,NaN,NaN
2006-12-16 19:00:00,NaN,NaN
2006-12-16 20:00:00,NaN,NaN
2006-12-16 21:00:00,NaN,NaN
2006-12-16 22:00:00,NaN,NaN
2006-12-16 23:00:00,NaN,NaN
2006-12-17 00:00:00,NaN,NaN
2006-12-17 01:00:00,NaN,NaN


In [9]:
hourly_df.dropna(inplace=True)
print(hourly_df.shape)


(34565, 16)


In [10]:
X = hourly_df.drop("Global_active_power", axis=1)
y = hourly_df["Global_active_power"]

print(X.shape, y.shape)


(34565, 15) (34565,)


In [11]:
split_index = int(len(hourly_df) * 0.8)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

print("Train:", X_train.shape)
print("Test:", X_test.shape)


Train: (27652, 15)
Test: (6913, 15)


In [12]:
X_train.describe()


,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,total_sub_metering,hour,day,month,weekday,gap_lag_1,gap_lag_24,gap_roll_mean_24,gap_roll_std_24
count,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000,27652.000000
mean,0.122182,240.738716,4.702589,1.158268,1.349673,6.361829,8.869769,11.501013,15.754303,6.398597,2.995552,1.109198,1.110381,1.109647,0.780170
std,0.067329,3.119159,3.890191,3.609689,4.336873,7.364793,10.446411,6.922336,8.830280,3.561813,2.000022,0.929170,0.930037,0.442965,0.296594
min,0.021033,225.834500,0.503333,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.124000,0.124000,0.172947,0.000000
25%,0.076767,238.980292,1.470000,0.000000,0.000000,0.000000,0.683333,6.000000,8.000000,3.000000,1.000000,0.333258,0.333325,0.818838,0.609024
50%,0.105033,240.895917,3.450000,0.000000,0.316667,1.616667,2.683333,12.000000,16.000000,6.000000,3.000000,0.808350,0.809417,1.096683,0.782040
75%,0.146500,242.694625,6.733333,0.000000,0.616667,14.266667,17.516667,18.000000,23.000000,10.000000,5.000000,1.613842,1.615942,1.362124,0.958372
max,0.774333,251.902000,28.383333,48.366667,46.433333,20.450000,74.233333,23.000000,31.000000,12.000000,6.000000,6.560533,6.560533,3.501724,1.830196


## Final Observations – Milestone 2

- Time-based features (hour, day, month, weekday) were extracted.
- Lag and rolling window features were created to capture temporal dependencies.
- Dataset was cleaned after feature engineering.
- Data is now fully prepared for machine learning model training.
